# **SCRIPT**

QUESTO SCRIPT CREA IN AUTOMATICO LA CARTELLA PER FARE TRAINING (TEST E VALIDATION)

In [ ]:
# === MOUNT DRIVE ===
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import random
import shutil

# === CONFIGURAZIONE ===
base_path = "/content/drive/MyDrive/Computer_Vision_&_Deep_Learning/DATASET/Datasets_Nostri/Dataset_Clean_GoodAnnotation"
images_path = os.path.join(base_path, "images")
labels_path = os.path.join(base_path, "labels_corrette")

# Percentuali split (modifica qui)
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

# === CONTROLLA COERENZA ===
assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-6, "La somma delle proporzioni deve essere 1.0"

# === OUTPUT PATHS ===
output_base = "/content/drive/MyDrive/Computer_Vision_&_Deep_Learning/DATASET/Training_Test_Datasets/Training_Test_GoodAnnotation"
train_img_out = os.path.join(output_base, "images/train")
val_img_out   = os.path.join(output_base, "images/val")
test_img_out  = os.path.join(output_base, "images/test")
train_lbl_out = os.path.join(output_base, "labels/train")
val_lbl_out   = os.path.join(output_base, "labels/val")
test_lbl_out  = os.path.join(output_base, "labels/test")

# === CREA CARTELLE DESTINAZIONE ===
for path in [train_img_out, val_img_out, test_img_out, train_lbl_out, val_lbl_out, test_lbl_out]:
    os.makedirs(path, exist_ok=True)

# === RACCOGLI E MESCOLA LE IMMAGINI ===
all_images = [f for f in os.listdir(images_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
all_images.sort()
random.shuffle(all_images)

# === CALCOLO SPLIT ===
total = len(all_images)
n_train = int(total * train_ratio)
n_val   = int(total * val_ratio)

train_images = all_images[:n_train]
val_images   = all_images[n_train:n_train + n_val]
test_images  = all_images[n_train + n_val:]

# === FUNZIONE DI COPIA ===
def copy_pairs(image_list, dst_img_dir, dst_lbl_dir):
    for img_file in image_list:
        base_name = os.path.splitext(img_file)[0]
        img_src = os.path.join(images_path, img_file)
        lbl_src = os.path.join(labels_path, base_name + ".txt")

        img_dst = os.path.join(dst_img_dir, img_file)
        lbl_dst = os.path.join(dst_lbl_dir, base_name + ".txt")

        if os.path.exists(img_src) and os.path.exists(lbl_src):
            shutil.copyfile(img_src, img_dst)
            shutil.copyfile(lbl_src, lbl_dst)
        else:
            print(f"⚠️ File mancante: {img_file} o {base_name}.txt")

# === COPIA FILE ===
copy_pairs(train_images, train_img_out, train_lbl_out)
copy_pairs(val_images, val_img_out, val_lbl_out)
copy_pairs(test_images, test_img_out, test_lbl_out)

# === LOG ===
print(f"Dataset YOLO Pose pronto!")
print(f"Train: {len(train_images)} immagini")
print(f"Val: {len(val_images)} immagini")
print(f"Test: {len(test_images)} immagini")
